In [1]:
import pandas as pd #needs to be version 0.23.4 for tsfresh to work

In [56]:
from os import listdir, path
from datautils import dir_to_dataframe

feature_dir = "data/features"
raw_features = dir_to_dataframe(feature_dir)

label_dir = "data/labels"
labels = dir_to_dataframe(label_dir)

In [54]:
raw_features.describe()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
count,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07
mean,8.184819e+11,1.161035e+01,1.689684e+02,6.914529e-02,4.468976e+00,8.942974e-01,-1.710847e-03,2.800733e-04,-2.569688e-04,3.803484e+03,9.006543e+00
std,4.952476e+11,8.691672e+01,1.072966e+02,1.424161e+00,8.130664e+00,3.251549e+00,1.445619e-01,3.400630e-01,1.480824e-01,1.436686e+06,8.106415e+00
min,0.000000e+00,7.500000e-01,0.000000e+00,-7.841969e+01,-7.299412e+01,-7.844842e+01,-4.845575e+01,-7.488861e+01,-5.355445e+01,0.000000e+00,-2.000000e+00
25%,3.779571e+11,3.900000e+00,7.800000e+01,-5.081238e-01,-2.022476e+00,-9.316101e-01,-2.678888e-02,-2.993851e-02,-1.876500e-02,2.410000e+02,1.020000e+00
50%,8.074539e+11,4.250000e+00,1.688700e+02,6.137085e-02,9.081485e+00,7.757450e-01,-6.432411e-04,2.645046e-04,-3.542004e-05,5.200000e+02,7.530000e+00
75%,1.254130e+12,8.000000e+00,2.629840e+02,6.350620e-01,9.709778e+00,2.750938e+00,2.330639e-02,3.143085e-02,1.823425e-02,8.630000e+02,1.548000e+01
max,1.709397e+12,6.070101e+03,3.599995e+02,6.687346e+01,7.505589e+01,7.805576e+01,3.983975e+01,8.031496e+01,6.630078e+01,1.495797e+09,1.480186e+02


In [11]:
len(labels)

20018

In [12]:
len(labels.drop_duplicates(subset = 'bookingID', keep = False))

19982

In [13]:
#drop all trips with duplicate labels since there are only 26 of them
deduplicated_labels = labels.drop_duplicates(subset = 'bookingID', keep = False).set_index(keys=['bookingID']).sort_index()

In [14]:
deduplicated_labels.head()

,label
bookingID,
0,0
1,1
2,1
4,1
6,0


In [38]:
sorted_df = raw_features.set_index(['bookingID', 'second']).sort_index()
sorted_df.describe()

,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,Speed
count,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07
mean,1.161035e+01,1.689684e+02,6.914529e-02,4.468976e+00,8.942974e-01,-1.710847e-03,2.800733e-04,-2.569688e-04,9.006543e+00
std,8.691672e+01,1.072966e+02,1.424161e+00,8.130664e+00,3.251549e+00,1.445619e-01,3.400630e-01,1.480824e-01,8.106415e+00
min,7.500000e-01,0.000000e+00,-7.841969e+01,-7.299412e+01,-7.844842e+01,-4.845575e+01,-7.488861e+01,-5.355445e+01,-2.000000e+00
25%,3.900000e+00,7.800000e+01,-5.081238e-01,-2.022476e+00,-9.316101e-01,-2.678888e-02,-2.993851e-02,-1.876500e-02,1.020000e+00
50%,4.250000e+00,1.688700e+02,6.137085e-02,9.081485e+00,7.757450e-01,-6.432411e-04,2.645046e-04,-3.542004e-05,7.530000e+00
75%,8.000000e+00,2.629840e+02,6.350620e-01,9.709778e+00,2.750938e+00,2.330639e-02,3.143085e-02,1.823425e-02,1.548000e+01
max,6.070101e+03,3.599995e+02,6.687346e+01,7.505589e+01,7.805576e+01,3.983975e+01,8.031496e+01,6.630078e+01,1.480186e+02


In [16]:
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
extracted_features = extract_features(raw_features, column_id="bookingID", column_sort="second", default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 30/30 [00:46<00:00,  1.50s/it]


In [17]:
extracted_features

variable,Accuracy__length,Accuracy__maximum,Accuracy__mean,Accuracy__median,Accuracy__minimum,Accuracy__standard_deviation,Accuracy__sum_values,Accuracy__variance,Bearing__length,Bearing__maximum,...,gyro_y__sum_values,gyro_y__variance,gyro_z__length,gyro_z__maximum,gyro_z__mean,gyro_z__median,gyro_z__minimum,gyro_z__standard_deviation,gyro_z__sum_values,gyro_z__variance
id,,,,,,,,,,,,,,,,,,,,,
0,1004.0,48.000,10.165339,8.0000,4.000,3.853977,10206.000,14.853141,1004.0,359.979767,...,-6.142259,0.010035,1004.0,0.372807,-0.004188,-0.003522,-0.731892,0.063654,-4.204491,0.004052
1,851.0,7.709,3.718763,3.9000,3.000,0.597582,3164.667,0.357104,851.0,337.000000,...,-6.416598,0.008399,851.0,0.235131,0.000405,-0.000384,-0.121265,0.033818,0.344877,0.001144
2,195.0,8.000,3.930626,3.6340,3.000,1.114485,766.472,1.242077,195.0,354.000000,...,-2.507892,0.013694,195.0,0.255884,0.002597,0.001764,-0.125907,0.036122,0.506505,0.001305
4,1094.0,10.000,10.000000,10.0000,10.000,0.000000,10940.000,0.000000,1094.0,353.855377,...,25.415907,0.012660,1094.0,0.505220,-0.000376,-0.001147,-0.348189,0.065897,-0.411573,0.004342
6,1095.0,12.000,4.586721,4.0040,3.000,1.328938,5022.460,1.766076,1095.0,359.000000,...,0.477217,0.011399,1095.0,0.364534,0.002930,0.002902,-0.348345,0.057411,3.207864,0.003296
7,949.0,3.900,3.681034,3.9000,3.000,0.377650,3493.301,0.142620,949.0,359.000000,...,3.249002,0.005778,949.0,0.532744,-0.005800,-0.003666,-0.852878,0.107804,-5.504609,0.011622
8,387.0,18.204,7.008253,6.0680,3.000,3.148948,2712.194,9.915875,387.0,348.000000,...,-0.268585,0.005957,387.0,0.070053,-0.001139,0.000168,-0.088150,0.020615,-0.440643,0.000425
10,375.0,9.000,3.815000,3.9000,3.000,0.845287,1430.625,0.714510,375.0,357.000000,...,0.866791,0.012611,375.0,0.259216,-0.001383,-0.000946,-0.134918,0.038527,-0.518509,0.001484
11,225.0,8.000,4.222360,3.9000,3.000,1.046713,950.031,1.095609,225.0,356.000000,...,-0.925812,0.023975,225.0,0.267076,0.003009,0.000739,-0.265598,0.071800,0.676991,0.005155


In [18]:
combined = extracted_features.merge(labels, left_index=True, right_on="bookingID", how="left")
x = combined.drop(['bookingID', 'label'], axis=1)
y = combined.label.values

In [19]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.33, shuffle=True, random_state=42)

In [59]:
x_train

,Accuracy__length,Accuracy__maximum,Accuracy__mean,Accuracy__median,Accuracy__minimum,Accuracy__standard_deviation,Accuracy__sum_values,Accuracy__variance,Bearing__length,Bearing__maximum,...,gyro_y__sum_values,gyro_y__variance,gyro_z__length,gyro_z__maximum,gyro_z__mean,gyro_z__median,gyro_z__minimum,gyro_z__standard_deviation,gyro_z__sum_values,gyro_z__variance
4109,598.0,699.999,4.749246,3.0000,3.000,28.571745,2840.049,816.344624,598.0,359.000000,...,10.857910,0.837584,598.0,5.635422,0.012059,-0.007309,-7.675507,0.660119,7.211563,0.435757
533,1242.0,2299.999,8.682907,4.0000,3.000,74.993229,10784.170,5623.984409,1242.0,359.000000,...,1.733723,0.015116,1242.0,0.452599,-0.000884,-0.000890,-0.475358,0.095610,-1.097585,0.009141
15813,1229.0,57.000,11.664768,8.0000,4.000,7.644666,14336.000,58.440915,1229.0,358.756592,...,23.628372,0.014399,1229.0,0.327463,0.002563,0.003953,-0.932339,0.048950,3.149382,0.002396
950,975.0,8.000,3.907576,3.9000,3.000,0.918629,3809.887,0.843879,975.0,348.000000,...,-1.143525,0.014614,975.0,0.797566,-0.003958,-0.002056,-0.361856,0.077873,-3.859355,0.006064
18378,269.0,8.000,3.947279,3.9000,3.000,1.037363,1061.818,1.076122,269.0,357.000000,...,-6.174556,0.014900,269.0,0.199125,0.002875,0.002182,-0.195843,0.038806,0.773338,0.001506
11926,735.0,16.000,7.907980,6.0000,3.000,4.508685,5812.365,20.328239,735.0,344.000000,...,-8.027276,0.180208,735.0,2.338329,-0.002432,-0.001896,-1.278604,0.127547,-1.787211,0.016268
14846,690.0,12.000,4.822580,4.4140,3.000,1.550775,3327.580,2.404904,690.0,359.000000,...,0.155893,0.027173,690.0,0.403354,-0.000968,-0.001039,-0.425590,0.050529,-0.668097,0.002553
14192,957.0,1000.000,26.888585,25.0000,9.268,33.209841,25732.376,1102.893551,957.0,354.000000,...,-4.762265,0.004494,957.0,0.321976,0.025829,0.023436,-0.296942,0.049109,24.718130,0.002412
13591,352.0,699.999,12.591781,4.0000,3.900,64.120318,4432.307,4111.415118,352.0,359.000000,...,2.354275,0.006036,352.0,0.176540,0.005692,0.002443,-0.129503,0.043813,2.003638,0.001920
13273,242.0,10.554,6.759351,6.5720,3.900,2.810455,1635.763,7.898659,242.0,354.524050,...,1.361664,0.138611,242.0,1.309235,0.015163,0.008926,-0.607056,0.225862,3.669464,0.051014


In [20]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(solver = 'lbfgs', max_iter=4000)
logistic_regression.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
# accuracy
logistic_regression.score(x_valid, y_valid)

0.7726309415682713

In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, logistic_regression.predict(x_valid))

0.558408642181468

In [24]:
logistic_regression_correlations = list(zip(x_valid.columns, logistic_regression.coef_[0]))
sorted(logistic_regression_correlations, key = lambda x: abs(x[1]), reverse = True)

[('acceleration_z__variance', 0.02598885659240507),
 ('acceleration_y__variance', 0.02158163847564987),
 ('acceleration_z__maximum', 0.0208026128460538),
 ('acceleration_y__minimum', -0.020080210755336152),
 ('Speed__median', -0.01972391044574833),
 ('Speed__mean', -0.01663917725088994),
 ('acceleration_z__minimum', -0.015506893239062607),
 ('Speed__maximum', -0.014464279549608921),
 ('acceleration_x__variance', 0.012075782230627483),
 ('acceleration_x__maximum', 0.011873841363977077),
 ('Bearing__minimum', -0.010426655662327824),
 ('acceleration_x__minimum', -0.010121439025531968),
 ('acceleration_y__maximum', 0.008121020713537298),
 ('gyro_y__maximum', 0.005693130207321187),
 ('gyro_z__minimum', -0.00510527827863682),
 ('acceleration_y__mean', -0.004808252508548729),
 ('acceleration_y__median', -0.00467581324245795),
 ('gyro_y__minimum', -0.004610552358276986),
 ('gyro_z__maximum', 0.004385017274144061),
 ('gyro_x__minimum', -0.004185030667402213),
 ('acceleration_z__standard_deviati

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_x_train = scaler.fit_transform(x_train)

scaled_x_valid = scaler.fit_transform(x_valid)

In [35]:
from sklearn.svm import SVC
svm = SVC(gamma='auto')
svm.fit(scaled_x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [36]:
svm.score(scaled_x_valid, y_valid)

0.7705116560702392

In [37]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, svm.predict(scaled_x_valid))

0.5437264864493188

In [52]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(72, 24, 72), random_state=1, max_iter=500)

mlpc.fit(scaled_x_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(72, 24, 72), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [53]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, mlpc.predict(scaled_x_valid))

0.5732040067581505